In [ ]:
!pip install transformers torch scikit-learn

In [ ]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import TensorDataset, DataLoader
from torch.nn import CrossEntropyLoss
from torch.optim import AdamW
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

In [ ]:
df = pd.read_csv('/content/emails.csv')  # Update path if needed

# Assume labels: 'spam' column (1 = spam, 0 = ham)
df['label'] = df['spam']  # Or adjust if needed
df['text'] = df['text'].astype(str).str.lower()

print(df.head())

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    df['text'],
    df['label'],
    test_size=0.2,
    stratify=df['label'],
    random_state=42
)

In [ ]:
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

In [ ]:
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
weights = torch.tensor(class_weights, dtype=torch.float)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
weights = weights.to(device)

loss_fn = CrossEntropyLoss(weight=weights)

In [ ]:
train_encodings = tokenizer(X_train.tolist(), truncation=True, padding=True, max_length=256)

train_dataset = TensorDataset(
    torch.tensor(train_encodings['input_ids']),
    torch.tensor(train_encodings['attention_mask']),
    torch.tensor(y_train.values)
)

train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)

In [ ]:
optimizer = AdamW(model.parameters(), lr=2e-5)

In [ ]:
model.train()

for epoch in range(2):  # Can increase for better performance
    for batch in train_dataloader:
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits

        loss = loss_fn(logits, labels)
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1} Loss: {loss.item()}")

In [ ]:
test_encodings = tokenizer(X_test.tolist(), truncation=True, padding=True, max_length=256)

input_ids = torch.tensor(test_encodings['input_ids']).to(device)
attention_mask = torch.tensor(test_encodings['attention_mask']).to(device)
labels = torch.tensor(y_test.values).to(device)

model.eval()
with torch.no_grad():
    outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    preds = torch.argmax(outputs.logits, dim=1)

y_pred = preds.cpu().numpy()
y_true = labels.cpu().numpy()

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
print("Accuracy:", accuracy_score(y_true, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_true, y_pred))
print("Classification Report:\n", classification_report(y_true, y_pred))

In [ ]:
model.save_pretrained("./fine_tuned_distilbert_email/")
tokenizer.save_pretrained("./fine_tuned_distilbert_email/")

!zip -r distilbert_email_model.zip fine_tuned_distilbert_email

In [ ]:
from google.colab import files
files.download('distilbert_email_model.zip')

In [ ]:
def predict_email(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=256)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    model.eval()
    with torch.no_grad():
        logits = model(**inputs).logits
        pred = torch.argmax(logits, dim=1).item()

    return "SPAM" if pred == 1 else "HAM"

print(predict_email("Congratulations! You've won a free trip. Click here to claim."))
print(predict_email("Hey John, please check the attached invoice and confirm."))